<a href="https://colab.research.google.com/github/Ash100/CADD_Project/blob/main/Desc_Ukn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# QSAR - Generation of Feature Profiles for a Dataset of Unknown Compounds

My name is **Dr. Ashfaq Ahmad** and I work in the field of Structure Biology and Bioinformatics. A step-by-step video demonstration can be found on [**Video Tutorial**](https://youtu.be/NyAiwGwIPCM)

These files are prepared for teaching and research purposes. If you want to use it for commercial purposes, please **contact us**.


**Quantitative structure–activity relationship (QSAR)** model(s) are generated by the users to test their compounds. Here we will learn how can we build a QSAR model for our data.

Next we will use that model to predict Activity of Unknown compounds. Once you generate and train your model, you can keep it and use in future.

I suggest you to read some literature from the field of your choice.

In [ ]:
#@title Install RDKit
!pip install rdkit-pypi
!pip install networkx

In [ ]:
#@title Import necessary libraries
import pandas as pd
import networkx as nx
from rdkit import Chem
from rdkit.Chem import Descriptors, rdMolDescriptors
from rdkit.Chem import Descriptors, rdmolops

In [ ]:
# Load data from a CSV file (make sure to upload your file to Colab)
# The CSV file should have at least two columns: "SMILES" and "Activity"
file_path = '/content/QSAR.csv'  # Change this to the path of your CSV file
data = pd.read_csv(file_path)

You CSV file should have two columns, labelled as **ID** and **SMILES**.

In [ ]:
#@title Lipinski's Descriptors for unknown compound datset
import pandas as pd
from rdkit import Chem
from rdkit.Chem import Descriptors

# Define functions to calculate Lipinski's descriptors

def lipinski_descriptors(mol):
    if mol is None:
        return None
    descriptors = {
        'MolecularWeight': Descriptors.MolWt(mol),
        'LogP': Descriptors.MolLogP(mol),
        'NumHDonors': Descriptors.NumHDonors(mol),
        'NumHAcceptors': Descriptors.NumHAcceptors(mol),
        'NumRotatableBonds': Descriptors.NumRotatableBonds(mol),
        'NumRings': Descriptors.RingCount(mol),
    }
    return descriptors

# Function to calculate all desired descriptors
def calculate_descriptors(smiles):
    mol = Chem.MolFromSmiles(smiles)
    if mol is None:
        return None
    descriptors = lipinski_descriptors(mol)
    return descriptors

# Calculate descriptors for each compound
descriptor_data = []
for index, row in data.iterrows():
    smiles = row['SMILES']
    id_ = row['ID']
    descriptors = calculate_descriptors(smiles)
    if descriptors is not None:
        descriptors['ID'] = id_
        descriptors['SMILES'] = smiles
        descriptor_data.append(descriptors)

# Convert the descriptor data to a DataFrame
descriptor_df = pd.DataFrame(descriptor_data)

# Save the results to a new CSV file
output_file_path = '/content/lipinski_descriptors_unknown.csv'  # Update this to your desired output path
descriptor_df.to_csv(output_file_path, index=False)

print(f"Descriptors calculated and saved to {output_file_path}")

In [ ]:
#@title Topological Descriptors for unknown compound dataset
import pandas as pd
import networkx as nx
from rdkit import Chem
from rdkit.Chem import Descriptors, rdmolops

# Define functions to calculate specific topological descriptors

def wiener_index(mol):
    g = nx.Graph(rdmolops.GetAdjacencyMatrix(mol))
    wiener_index = nx.wiener_index(g)
    return wiener_index

def zagreb_index(mol):
    adj_matrix = rdmolops.GetAdjacencyMatrix(mol)
    degrees = adj_matrix.sum(axis=0)
    zagreb_index = sum(degrees**2)
    return zagreb_index

def tpsa(mol):
    return Descriptors.TPSA(mol)

def kier_hall_chi_indices(mol):
    return {
        'Kappa1': Descriptors.Kappa1(mol),
        'Kappa2': Descriptors.Kappa2(mol),
        'Kappa3': Descriptors.Kappa3(mol),
    }

def balaban_index(mol):
    return Descriptors.BalabanJ(mol)

# Function to calculate all desired descriptors
def calculate_topological_descriptors(smiles):
    mol = Chem.MolFromSmiles(smiles)
    if mol is None:
        return None
    descriptors = {
        'WienerIndex': wiener_index(mol),
        'ZagrebIndex': zagreb_index(mol),
        'TPSA': tpsa(mol),
        'BalabanIndex': balaban_index(mol)
    }
    kier_hall = kier_hall_chi_indices(mol)
    descriptors.update(kier_hall)
    return descriptors

# Calculate topological descriptors for each compound
descriptor_data = []
for index, row in data.iterrows():
    smiles = row['SMILES']
    id_ = row['ID']
    descriptors = calculate_topological_descriptors(smiles)
    if descriptors is not None:
        descriptors['ID'] = id_
        descriptors['SMILES'] = smiles
        descriptor_data.append(descriptors)

# Convert the descriptor data to a DataFrame
descriptor_df = pd.DataFrame(descriptor_data)

# Save the results to a new CSV file
output_file_path = '/content/topological_descriptors_unknown.csv'  # Update this to your desired output path
descriptor_df.to_csv(output_file_path, index=False)

print(f"Descriptors calculated and saved to {output_file_path}")

In [ ]:
#@title Calculate QED Score for Unknow dataset
import pandas as pd
from rdkit import Chem
from rdkit.Chem import QED

# Function to calculate QED score
def qed_score(smiles):
    mol = Chem.MolFromSmiles(smiles)
    if mol is None:
        return None
    return QED.qed(mol)

# Calculate QED score for each compound
qed_data = []
for index, row in data.iterrows():
    smiles = row['SMILES']
    id_ = row['ID']
    qed_value = qed_score(smiles)
    if qed_value is not None:
        qed_data.append({'ID': id_, 'SMILES': smiles, 'QED': qed_value})

# Convert the QED data to a DataFrame
qed_df = pd.DataFrame(qed_data)

# Save the results to a new CSV file
output_file_path = '/content/qed_scores_unknown.csv'  # Update this to your desired output path
qed_df.to_csv(output_file_path, index=False)

print(f"QED scores calculated and saved to {output_file_path}")

The last one is the calculation of 3D descriptors. It is a heavy job therefore, ensure your number of compounds and click the corresponding **cell**. Remember some of the compounds might failed because of several reasons.
a. Forcefield doesn't suits well
b. SMILES contains salts (SMILES without Salts are treated well by rdkit).

If your data is below 5000 compounds, just run the **Cell001**. Incase your compounds are more than 5000, then please try **Cell002**. This cell contains some changes, please note it below;

**Cell001**: Simplest Script


**Cell002**: It is not the simple script and it includes chunk strategy. It means you will need to alter the chunk value..as a result all the compounds will not be loaded to the memory at once and you will achieve better speed. Look the code and change the chunk value (appears two time). Also this script, asks rdkit to try different ff incase it fails in first attempt, also it allows higher number of iterations, inorder to rescue the file.

In [ ]:
#@title Cell001: 3D Descriptor Calculations for Unknown
import pandas as pd
from rdkit import Chem
from rdkit.Chem import AllChem, Descriptors
from rdkit.Chem import rdMolDescriptors

# Function to generate 3D conformer and optimize
def generate_3D_conformer(mol):
    if mol is None:
        return None
    # Add hydrogens
    mol = Chem.AddHs(mol)
    # Generate 3D coordinates
    AllChem.EmbedMolecule(mol, randomSeed=42)
    # Minimize energy
    AllChem.MMFFOptimizeMolecule(mol, maxIters=500, nonBondedThresh=500.0)
    return mol

# Define functions to calculate 3D descriptors

def molecular_volume(mol):
    # Calculate molecular volume
    conformer = mol.GetNumConformers()
    if conformer == 0:
        mol = generate_3D_conformer(mol)
    return rdMolDescriptors.CalcExactMolWt(mol)  # Placeholder; replace with actual volume calculation if available

def surface_area(mol):
    # Calculate surface area
    conformer = mol.GetNumConformers()
    if conformer == 0:
        mol = generate_3D_conformer(mol)
    return rdMolDescriptors.CalcExactMolWt(mol)  # Placeholder; replace with actual surface area calculation if available

def radius_of_gyration(mol):
    # Calculate radius of gyration
    conformer = mol.GetNumConformers()
    if conformer == 0:
        mol = generate_3D_conformer(mol)
    return rdMolDescriptors.CalcRadiusOfGyration(mol)

# Function to calculate all desired 3D descriptors
def calculate_3D_descriptors(smiles):
    mol = Chem.MolFromSmiles(smiles)
    if mol is None:
        return None
    mol = generate_3D_conformer(mol)
    descriptors = {
        'MolecularVolume': molecular_volume(mol),
        'SurfaceArea': surface_area(mol),
        'RadiusOfGyration': radius_of_gyration(mol),
    }
    return descriptors

# Calculate 3D descriptors for each compound
descriptor_data = []
for index, row in data.iterrows():
    smiles = row['SMILES']
    id_ = row['ID']
    descriptors = calculate_3D_descriptors(smiles)
    if descriptors is not None:
        descriptors['ID'] = id_
        descriptors['SMILES'] = smiles
        descriptor_data.append(descriptors)

# Convert the descriptor data to a DataFrame
descriptor_df = pd.DataFrame(descriptor_data)

# Save the results to a new CSV file
output_file_path = '/content/3D_descriptors_unknown.csv'  # Update this to your desired output path
descriptor_df.to_csv(output_file_path, index=False)

print(f"3D Descriptors calculated and saved to {output_file_path}")

In [ ]:
#@title Cell002: 3D Descriptor Calculations for Unknown
import pandas as pd
from rdkit import Chem
from rdkit.Chem import AllChem, Descriptors
from rdkit.Chem import rdMolDescriptors
import rdkit.Chem.rdFreeSASA as FreeSASA
from multiprocessing import Pool, cpu_count
from tqdm import tqdm

# Function to generate 3D conformer and optimize
def generate_3D_conformer(mol):
    if mol is None:
        return None
    # Add hydrogens
    mol = Chem.AddHs(mol)
    try:
        # Increase embedding attempts and set more parameters
        params = AllChem.ETKDGv2()
        params.randomSeed = 42
        if AllChem.EmbedMolecule(mol, params=params) != 0:
            raise ValueError("Embedding failed with MMFF")
        # Minimize energy using MMFF
        if AllChem.MMFFOptimizeMolecule(mol, maxIters=2000, nonBondedThresh=1000.0) != 0:
            raise ValueError("MMFF Optimization failed")
    except Exception as e:
        print(f"Error generating 3D conformer with MMFF: {e}")
        try:
            # Try with UFF if MMFF fails
            if AllChem.EmbedMolecule(mol, params=params) != 0:
                raise ValueError("Embedding failed with UFF")
            # Minimize energy using UFF
            if AllChem.UFFOptimizeMolecule(mol, maxIters=2000) != 0:
                raise ValueError("UFF Optimization failed")
        except Exception as e:
            print(f"Error generating 3D conformer with UFF: {e}")
            return None
    return mol

# Define functions to calculate 3D descriptors
def molecular_volume(mol):
    if mol.GetNumConformers() == 0:
        mol = generate_3D_conformer(mol)
    if mol is None:
        return None
    return rdMolDescriptors.CalcExactMolWt(mol)  # Placeholder; replace with actual volume calculation if available

def surface_area(mol):
    if mol.GetNumConformers() == 0:
        mol = generate_3D_conformer(mol)
    if mol is None:
        return None
    radii = FreeSASA.classifyAtoms(mol)
    return FreeSASA.CalcSASA(mol, radii)

def radius_of_gyration(mol):
    if mol.GetNumConformers() == 0:
        mol = generate_3D_conformer(mol)
    if mol is None:
        return None
    return rdMolDescriptors.CalcRadiusOfGyration(mol)

def calculate_3D_descriptors(smiles):
    mol = Chem.MolFromSmiles(smiles)
    if mol is None:
        return None
    mol = generate_3D_conformer(mol)
    if mol is None:
        return None
    descriptors = {
        'MolecularVolume': molecular_volume(mol),
        'SurfaceArea': surface_area(mol),
        'RadiusOfGyration': radius_of_gyration(mol),
    }
    return descriptors

# Function to process each row
def process_row(row):
    smiles = row['SMILES']
    id_ = row['ID']
    descriptors = calculate_3D_descriptors(smiles)
    if descriptors is not None:
        descriptors['ID'] = id_
        descriptors['SMILES'] = smiles
        return descriptors
    else:
        print(f"Failed to generate 3D conformer for SMILES: {smiles}")
        return None

# Function to process data in chunks
def process_data_in_chunks(input_file_path, chunk_size=5000):
    chunk_list = []
    for chunk in pd.read_csv(input_file_path, chunksize=chunk_size):
        with Pool(cpu_count()) as p:
            # Wrap chunk processing with tqdm to show a progress bar
            descriptor_data = list(tqdm(p.imap(process_row, [row for _, row in chunk.iterrows()]), total=len(chunk), desc="Processing Chunk"))
            chunk_list.extend([d for d in descriptor_data if d is not None])
    return pd.DataFrame(chunk_list)

# Save the results to a new CSV file
output_file_path = '/content/3D_descriptors.csv'  # Update this to your desired output path
descriptor_df.to_csv(output_file_path, index=False)

print(f"3D Descriptors calculated and saved to {output_file_path}")

In [ ]:
#@title Merge all the CSVs into a One master file
import pandas as pd

# Define file paths for the input CSV files
qed_file = '/content/qed_scores.csv'
three_d_file = '/content/3D_descriptors (1).csv'
lipinski_file = '/content/lipinski_descriptors (1).csv'
descriptors_file = '/content/Topological_descriptors.csv'

# Read the CSV files into DataFrames
df_qed = pd.read_csv(qed_file)
df_3d = pd.read_csv(three_d_file)
df_lipinski = pd.read_csv(lipinski_file)
df_descriptors = pd.read_csv(descriptors_file)

# Merge DataFrames on the 'SMILES' column
# You can use 'outer' merge to ensure all rows are included, even if they don't match in all files
merged_df = pd.merge(df_qed, df_3d, on='SMILES', how='outer')
merged_df = pd.merge(merged_df, df_lipinski, on='SMILES', how='outer')
merged_df = pd.merge(merged_df, df_descriptors, on='SMILES', how='outer')

# Save the merged DataFrame to a new CSV file
output_file_path = '/content/merged_descriptors_unknown.csv'
merged_df.to_csv(output_file_path, index=False)

print(f"CSV files merged and saved to {output_file_path}")